In [1]:
import json

from tqdm import tqdm
import torch
from ltp import LTP

In [2]:
from collections import Counter

In [3]:
ltp = LTP(path='base')
ltp1 = LTP(path='base1')
ltp2 = LTP(path='base2')

In [8]:
text = '乔丹 是 我 一直 完全 非常 喜欢 的 一 个 球星'

seg, hidden = ltp2.seg([text.split(' ')], is_preseged=True)
sdp = ltp2.sdp(hidden, mode='mix')

In [9]:
sdp

[[(1, 2, 'EXP'),
  (2, 0, 'Root'),
  (3, 7, 'AGT'),
  (4, 7, 'mDEPD'),
  (5, 7, 'mDEPD'),
  (6, 7, 'mDEPD'),
  (7, 11, 'rDATV'),
  (8, 7, 'mDEPD'),
  (9, 10, 'MEAS'),
  (10, 11, 'MEAS'),
  (11, 2, 'LINK')]]

In [11]:
print(sorted(ltp.sdp_vocab))

['AGT',
 'CONT',
 'DATV',
 'EXP',
 'FEAT',
 'LINK',
 'LOC',
 'MANN',
 'MATL',
 'MEAS',
 'PAT',
 'REAS',
 'Root',
 'SCO',
 'STAT',
 'TIME',
 'TOOL',
 'dAGT',
 'dCONT',
 'dDATV',
 'dEXP',
 'dFEAT',
 'dLINK',
 'dLOC',
 'dMANN',
 'dMATL',
 'dMEAS',
 'dPAT',
 'dREAS',
 'dSCO',
 'dSTAT',
 'dTIME',
 'dTOOL',
 'eCOO',
 'ePREC',
 'eSUCC',
 'mDEPD',
 'mNEG',
 'mPUNC',
 'mRELA',
 'rAGT',
 'rCONT',
 'rDATV',
 'rEXP',
 'rFEAT',
 'rLINK',
 'rLOC',
 'rMANN',
 'rMATL',
 'rMEAS',
 'rPAT',
 'rREAS',
 'rSCO',
 'rSTAT',
 'rTIME',
 'rTOOL']

In [4]:
'''
# relationship
# head, rel, tail 
# head or tail: {utterance idx}-{word idx}
# example: (1-0, root, 1-3)
'''

'\n# relationship\n# head, rel, tail \n# head or tail: {utterance idx}-{word idx}\n# example: (1-0, root, 1-3)\n'

In [7]:
def pre_annot():
    for name in ['train']:
        file = f'data/{name}.json'
        out = f'pre_annot/{name}.json'
        fp = open(file, 'r', encoding='utf-8')
        fw = open(out, 'w', encoding='utf-8')
        data = json.load(fp)
        
        for i, d in enumerate(tqdm(data)):
            # error in val
            if i == 37:
                continue
            # error in train
            if i == 480:
                continue
            
            for turn, dialog in enumerate(d.get('dialog')):
                speaker = dialog.get('speaker')

                utterance = dialog.get('utterance')
                
                seg, hidden = ltp.seg([utterance.split(' ')], is_preseged=True)
                sdp = ltp.sdp(hidden, mode='mix')
                sdp1 = sdp[0]
                
                seg, hidden = ltp1.seg([utterance.split(' ')], is_preseged=True)
                sdp = ltp1.sdp(hidden, mode='mix')
                sdp2 = sdp[0]
                
                seg, hidden = ltp2.seg([utterance.split(' ')], is_preseged=True)
                sdp = ltp2.sdp(hidden, mode='mix')
                sdp3 = sdp[0]
                
                sdp = list([str(x) for x in sdp1]) + list([str(x) for x in sdp2]) + list([str(x) for x in sdp3])
                cnt = Counter(sdp)
                
                for tri, num in cnt.items():
                    if num < 3:
                        continue
                    tri = eval(tri)
                    head = f"{turn}-{tri[1]}"
                    tail = f"{turn}-{tri[0]}"
                    rel = tri[2]
                    
                    d['relationship'].append([head, rel, tail])
        
        save_str = json.dumps(data, ensure_ascii=False, indent=4, separators=(',', ': '))
        fw.write(save_str)
        fp.close()
        fw.close()

In [8]:
pre_annot()

100%|██████████| 783/783 [07:34<00:00,  1.72it/s]
